# ***Capstone Project Analisis Media Sosial - Group G***

| NIM | Nama | Deskripsi |
| -------- | -------- | -------- |
|1122094000006  |AHMAD IZZA |Ambil Data, Preprocessing, Visualisasi |
|11220940000047  |RAJWAA WARDA YUNENDA PUTRI |Ambil Data,Visualisasi, PPT |
|11220940000063   |AWALIA DAMAYANTI  |Ambil Data, Visualisasi, PPT, Video |
|11220940000073  |NIKEN SAFIRA |Ambil Data, Visualisasi |






## Latar Belakang

Pada bulan Mei 2025, program barak militer bagi siswa bermasalah di Jawa Barat yang digagas oleh Dedi Mulyadi menjadi perbincangan hangat di ruang publik. Program ini bertujuan membina kedisiplinan dan karakter siswa yang dianggap memiliki perilaku menyimpang melalui pendekatan militeristik.

Meskipun dinilai sebagai langkah tegas untuk mengatasi kenakalan remaja, program ini menimbulkan pro dan kontra. Sebagian masyarakat mendukungnya sebagai solusi konkret terhadap masalah perilaku siswa, sementara yang lain mengkritiknya karena dianggap terlalu keras dan berpotensi melanggar hak anak.

Diskusi mengenai program ini ramai terjadi di media sosial, khususnya YouTube, yang menjadi wadah bagi masyarakat untuk mengekspresikan opini mereka. Analisis sentimen terhadap percakapan ini dapat memberikan gambaran mengenai persepsi publik, serta menjadi masukan penting bagi pembuat kebijakan dalam menangani siswa bermasalah secara lebih bijak dan manusiawi.

## Rumusan Masalah

**Permasalahan**

1. Apa isu-isu utama yang muncul terkait program barak militer bagi siswa bermasalah di Jawa Barat?

2. Bagaimana sentimen publik terhadap program barak militer bagi siswa bermasalah?

3. Bagaimana pola percakapan warganet tentang program ini di media sosial, khususnya YouTube?

4. Bagaimana respons warganet terhadap sosok Dedi Mulyadi sebagai penggagas program tersebut?


**Periode Pengumpulan Data**

2 Mei-27 Mei 2025

**Sumber Data**

YouTube

**Kata Kunci**
* Program barak militer
* Barak Militer Dedi Mulyadi
* Barak militer anak
* Barak militer siswa
* Siswa nakal Jawa barat


## Packages

In [1]:
!pip install sastrawi
!pip install emoji # Install the emoji library
!pip install textblob
from textblob import TextBlob
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import emoji

nltk.download('popular')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 11.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    | 

True

# **Mempersiapkan Data**

## Pengambilan Data YouTube

Setelah mengambil data di YouTube lalu kami lakukan proses penggabungan dan duplikat agar satu keyword mengambil video yang berbeda dari yang lain.


```
import pandas as pd
import glob

# Mencari semua file CSV di dalam folder 'tweets-data'
csv_files = glob.glob('tweets-data/*.csv') # list nama file yang disimpan
csv_files.sort()  # Use the sort() method of the list object

n = 0
for filename in csv_files: ## Menggabungkan file csv dalam bentuk dataframe
  # Read the CSV file into a pandas DataFrame
  if n == 0:
    df = pd.read_csv(filename, delimiter=",")
    n = 1
  else:
    df2 = pd.read_csv(filename, delimiter=",")
    df = pd.concat([df, df2])
    n+=1

# Membuang duplikat berdasarkan id_str
df.drop_duplicates(subset=['id_str'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

# Ekspor Data Hasil Scrapping
from google.colab import files
namafile = "DATAX_Naturalisasi_fix.csv"
df.to_csv(namafile, index=False)
files.download(namafile)
"Exported"
```



Kami menggunakan 5 keyword dan melakukan proses pengambilan komentar di video youtobe secara acak menggunakan API




```
# Pengambilan yang kami lakukan
import csv
from datetime import datetime
import googleapiclient.discovery

api_key = "your-api-key"

keywords = [
    "Program barak militer",
    "Barak Militer Dedi Mulyadi",
    "Barak militer anak",
    "Barak militer siswa",
    "Siswa nakal Jawa barat"
]

used_video_ids = set()  # Untuk menyimpan video yang sudah dipakai

def search_and_get_comments(search_query, max_videos=10, max_comments_per_video=250, start_date=None, end_date=None, csv_filename="youtube_comments.csv"):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

    search_response = youtube.search().list(
        part="snippet",
        q=search_query,
        type="video",
        maxResults=30  # ambil lebih banyak untuk jaga-jaga (nanti kita filter jadi 10 unik)
    ).execute()

    # Filter video yang belum dipakai
    video_titles_and_ids = {}
    for item in search_response['items']:
        if item['id']['kind'] == 'youtube#video':
            video_id = item['id']['videoId']
            if video_id not in used_video_ids:
                video_titles_and_ids[video_id] = item['snippet']['title']
                if len(video_titles_and_ids) >= max_videos:
                    break

    used_video_ids.update(video_titles_and_ids.keys())  # Tandai video ini sudah terpakai

    if not video_titles_and_ids:
        print(f"No new videos found for keyword '{search_query}'. Skipping.")
        return

    total_comments = 0
    unique_comments = set()

    start_dt = datetime.strptime(start_date, "%Y-%m-%d") if start_date else None
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") if end_date else None

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["keyword", "video_id", "video_title", "comment_text", "published_at"])  # Header

        for video_id, video_title in video_titles_and_ids.items():
            try:
                next_page_token = None
                comments_written = 0

                while True:
                    request = youtube.commentThreads().list(
                        part="snippet",
                        videoId=video_id,
                        maxResults=100,
                        pageToken=next_page_token
                    )
                    response = request.execute()

                    for item in response['items']:
                        try:
                            snippet = item['snippet']['topLevelComment']['snippet']
                            comment_text = snippet['textDisplay']
                            published_at = snippet['publishedAt']
                            comment_dt = datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ")

                            if (start_dt and comment_dt < start_dt) or (end_dt and comment_dt > end_dt):
                                continue

                            unique_id = f"{video_id}_{comment_text}"
                            if unique_id in unique_comments:
                                continue
                            unique_comments.add(unique_id)

                            writer.writerow([search_query, video_id, video_title, comment_text, published_at])
                            total_comments += 1
                            comments_written += 1

                            if comments_written >= max_comments_per_video:
                                break
                        except KeyError:
                            continue

                    next_page_token = response.get("nextPageToken")
                    if not next_page_token or comments_written >= max_comments_per_video:
                        break

            except Exception as e:
                print(f"Error retrieving comments for video ID {video_id}: {e}")

    print(f"CSV file '{csv_filename}' created for keyword '{search_query}'.")
    print(f"Total unique comments retrieved: {total_comments}")
    print(f"Number of videos fetched: {len(video_titles_and_ids)}")

# Run for all keywords
start_date = "2025-05-02"
end_date = "2025-05-28"

for keyword in keywords:
    csv_file = f"youtube_comments_{keyword.replace(' ', '_')}.csv"
    search_and_get_comments(keyword, max_videos=10, max_comments_per_video=250, start_date=start_date, end_date=end_date, csv_filename=csv_file)

     
```




## Import Data

Berikut Link hasil pengambilan data dari youtobe:
- https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/main/all_youtube_comments_combined.csv


In [2]:
# Import data hasil pengambilan dari youtobe
import pandas as pd

# URL file raw CSV dari GitHub
url = 'https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/main/all_youtube_comments_combined.csv'

# Membaca file langsung dari URL
dataRaw = pd.read_csv(url, low_memory=False, encoding='utf8')

# Menampilkan beberapa baris pertama
dataRaw.head()

,keyword,video_id,video_title,comment_text,published_at
0,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,2025-05-21T10:53:25Z
1,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2025-05-21T00:28:40Z
2,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2025-05-18T11:55:57Z
3,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",2025-05-17T23:56:01Z
4,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",2025-05-17T16:44:17Z


In [3]:
# melihat kolom di dataRaw
dataRaw.columns

Index(['keyword', 'video_id', 'video_title', 'comment_text', 'published_at'], dtype='object')

## Preprocessing

### Duplikasi Data

In [4]:
# Menampilkan domment text yang duplikat
duplicate_count = dataRaw.duplicated(subset='comment_text').sum()
print(f"Jumlah komentar duplikat: {duplicate_count}")

Jumlah komentar duplikat: 86


In [5]:
# Menampilkan contoh komentar duplikat dalam bentuk tabel
dataRaw[dataRaw.duplicated(subset='comment_text', keep=False)].head(86)

,keyword,video_id,video_title,comment_text,published_at
0,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,2025-05-21T10:53:25Z
84,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Jujur Saya warga JAWA TIMUR merasa iri ! Pak P...,2025-05-04T05:09:55Z
124,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Setuju,2025-05-03T04:16:40Z
153,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Kok seperti gaya negara sovyet waktu perang di...,2025-05-02T13:56:29Z
225,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Bravo KDM,2025-05-02T08:06:55Z
...,...,...,...,...,...
4382,Barak militer anak,KmyfO5YZkdQ,Dedi Mulyadi Siapkan Rp6 Miliar untuk Kirim An...,Kami warga jakarta bisa nitip anak sekolah ngg...,2025-05-09T03:05:49Z
4429,Barak militer anak,UZ8USYlGZSs,Setop Kirim Anak ke Barak Militer,"KPAI PUSAT TOLONG YG DI JABAR, ANAK2 ISTIMEWAN...",2025-05-26T14:21:07Z
4472,Barak militer anak,UZ8USYlGZSs,Setop Kirim Anak ke Barak Militer,Program bagus,2025-05-12T21:12:21Z
4479,Barak militer anak,UZ8USYlGZSs,Setop Kirim Anak ke Barak Militer,Trus apa gunanya ada guru BP ?<br><br>Gak usah...,2025-05-12T06:46:33Z


In [6]:
# Menghapus Duplikat
dataRaw.drop_duplicates(subset='comment_text', inplace=True)

In [7]:
# Cek kembali yang Duplikat
duplicate_count = dataRaw.duplicated(subset='comment_text').sum()
print(f"Jumlah komentar duplikat: {duplicate_count}")

Jumlah komentar duplikat: 0


In [8]:
# Menyimpan dalam bentuk CSV untuk data tanpa komen yanng duplikat
# dataRaw.to_csv('comments_no_duplicates.csv', index=False)

### Menambahkan labelling ke dalam Dataset

Dari data komentar tanpa duplikat, dilanjutkan dengan labelling dan disimpan di link github berikut :

https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/youtube_comments_labeling_no%20duplicates_no%20comment%20judol.csv



Kami telah melakukan proses pelabelan sentimen (1:Positif, 2:Netral, dan 3:Negatif) pada setiap tweet sebelum data ini diunggah ke GitHub dan diimpor ke sini. Langkah ini dilakukan untuk mendukung analisis sentimen publik terkait topik Persepsi Publik di Media Sosial terhadap Program Barak Militer bagi Siswa Bermasalah di Jawa Barat.

In [9]:
url2 = "https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/youtube_comments_labeling_no%20duplicates_no%20comment%20judol.csv"
data = pd.read_csv(url2, low_memory=False, encoding='utf8')
data

,keyword,video_id,video_title,comment_text,published_at,label
0,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,2025-05-21T10:53:25Z,1
1,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2025-05-21T00:28:40Z,2
2,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2025-05-18T11:55:57Z,2
3,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",2025-05-17T23:56:01Z,1
4,Siswa nakal Jawa barat,3lQKfSWPHgA,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",2025-05-17T16:44:17Z,1
...,...,...,...,...,...,...
6365,Program barak militer,W412jxEtglo,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,2025-05-04T14:01:16Z,1
6366,Program barak militer,W412jxEtglo,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2025-05-04T14:00:50Z,2
6367,Program barak militer,W412jxEtglo,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,2025-05-04T14:00:36Z,1
6368,Program barak militer,W412jxEtglo,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2025-05-04T13:54:09Z,2


In [10]:
# Drop kolom yang tidak diperlukan
data = data.drop(columns=['keyword', 'video_id'])
# Ubah urutan kolom: published_at di awal, sisanya diikuti
cols = ['published_at', 'video_title', 'comment_text', 'label']
data = data[cols]

data

,published_at,video_title,comment_text,label
0,2025-05-21T10:53:25Z,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1
1,2025-05-21T00:28:40Z,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2
2,2025-05-18T11:55:57Z,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2
3,2025-05-17T23:56:01Z,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1
4,2025-05-17T16:44:17Z,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1
...,...,...,...,...
6365,2025-05-04T14:01:16Z,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1
6366,2025-05-04T14:00:50Z,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2
6367,2025-05-04T14:00:36Z,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1
6368,2025-05-04T13:54:09Z,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2


### Ubah Tipe Data

In [11]:
# Cek tipe data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6370 entries, 0 to 6369
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   published_at  6370 non-null   object
 1   video_title   6370 non-null   object
 2   comment_text  6370 non-null   object
 3   label         6370 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 199.2+ KB


In [12]:
# Ubah 'label' menjadi kategori
data['label'] = data['label'].astype('category')

# Ubah 'published_at' menjadi datetime (format ISO 8601 ke waktu biasa)
data['published_at'] = pd.to_datetime(data['published_at'], format='%Y-%m-%dT%H:%M:%SZ')

# Cek hasil
data.dtypes

,0
published_at,datetime64[ns]
video_title,object
comment_text,object
label,category


In [13]:
# Cek jumlah tiap label
data['label'].value_counts()

,count
label,
2,3411
1,2833
3,126


### Missing Value

In [14]:
# Cek Missing Value
data.isnull().sum()

,0
published_at,0
video_title,0
comment_text,0
label,0


Karena tidak ada missing value maka tidak perlu dihapus dan tidak perlu diubah apa apa datanya

# **Cleaning Text**

In [15]:
# Membersihkan teks dari URL, mention, emoji, hashtag, angka, tanda baca, dan spasi berlebih
import re
import string
import unicodedata

def clean_text(text):
    text = text.lower()  # Ubah ke lowercase

    # Hilangkan karakter aneh (emoji, simbol asing)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Hapus URL, mention, hashtag
    text = re.sub(r'http\S+|www.\S+', '', text)       # URL
    text = re.sub(r'@\w+', '', text)                  # mention
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'#\w+', '', text)                  # hashtag

    # Hapus angka
    text = re.sub(r'\d+', '', text)

    # Hapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [16]:
# Membersihkan semua teks dalam kolom comment_text
data['cleaned_text'] = data['comment_text'].apply(clean_text)
data

,published_at,video_title,comment_text,label,cleaned_text
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...
...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...


## Normalisasi Data

In [17]:
# Buat folder data
!mkdir -p data
!wget -P data/ https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/slangs%20%26%20typos.txt

# Lokasi file
slang_file = 'data/slangs & typos.txt'

# Membuat kamus slang dari file
slang_dict = {}
with open(slang_file, 'r', encoding='utf-8') as file:
    for line in file:
        if ':' in line:
            slang, standard = line.strip().split(':', 1)
            slang_dict[slang.strip()] = standard.strip()

# Pastikan sudah import word_tokenize (dari NLTK)
from nltk.tokenize import word_tokenize

# Fungsi untuk mengganti slang dengan kata baku
def normalize_slang(text):
    words = word_tokenize(text.lower())
    return ' '.join([slang_dict.get(word, word) for word in words])

# Terapkan ke kolom 'cleaned_text'
data['normalized_comment'] = data['cleaned_text'].apply(normalize_slang)

# Lihat hasil
data[['cleaned_text', 'normalized_comment']].head()

--2025-06-05 14:48:53--  https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/slangs%20%26%20typos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61847 (60K) [text/plain]
Saving to: ‘data/slangs & typos.txt’

slangs & typos.txt  100%[===================>]  60.40K  --.-KB/s    in 0.01s   

2025-06-05 14:48:54 (4.75 MB/s) - ‘data/slangs & typos.txt’ saved [61847/61847]



,cleaned_text,normalized_comment
0,program bagus,program bagus
1,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...
2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...
3,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...
4,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...


In [18]:
# Cek isi data
data

,published_at,video_title,comment_text,label,cleaned_text,normalized_comment
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus,program bagus
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...
...,...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...,ini manusia tidak berguna kamu bisanya mengala...
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...,jadi anak nakal biarin saja kata yang tidak se...
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...,kalau anak anak ini di siksa boleh ibu menangi...
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...,sudah lah hesti ngapai nagisin anak orang sila...


## Cleaning Data Lanjutan

In [19]:
# Membersihkan semua teks dalam kolom normalized_comment
data['cleaned_text2'] = data['normalized_comment'].apply(clean_text)
data

,published_at,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus,program bagus,program bagus
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...,gubernur jawa tengah tidak mikirin masa depan ...
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan saja dewan perwakilan rakyat daerah d...
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya anak tidak punya masalah tapi ibunya ...
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...,siapa tahu akan gedenya jadi tentara nasional ...
...,...,...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...,ini manusia tidak berguna kamu bisanya mengala...,ini manusia tidak berguna kamu bisanya mengala...
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...,jadi anak nakal biarin saja kata yang tidak se...,jadi anak nakal biarin saja kata yang tidak se...
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...,kalau anak anak ini di siksa boleh ibu menangi...,kalau anak anak ini di siksa boleh ibu menangi...
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...,sudah lah hesti ngapai nagisin anak orang sila...,sudah lah hesti ngapai nagisin anak orang sila...


### Stopword

Sebelumnya kita sudah gabungkan semua stopword menjadi satu file dalam link berikut :

https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/stopwords.txt

In [20]:
!wget -O data/stopwords_id.txt https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/stopwords.txt
# Menghapus stopwords bahasa Indonesia
# Baca file stopwords
with open('data/stopwords_id.txt', "r", encoding="utf-8", errors='replace') as file:
    id_stop = file.readlines()

# Bersihkan stopwords (strip newline/spasi & lowercase)
id_stop = [word.strip().lower() for word in id_stop]

# Tambahkan kata-kata tambahan
additional_stopwords = {
    'nya', 'iya', 'itu', 'kah', 'dan', 'ke', 'di',
    'turun', 'naik', 'ambil', 'lihat',
    'tolong', 'atas', 'pokok', 'hari',
    'naikin', 'presiden', 'indonesia', 'sih', 'pakai',
    'biar', 'juta', 'ribu',
}

# Gabungkan ke list utama
id_stop = set(id_stop).union(additional_stopwords)

# Fungsi hapus stopwords
def remove_custom_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in id_stop])

# Terapkan ke kolom yang sudah dibersihkan sebelumnya
data['no_stopwords'] = data['cleaned_text2'].apply(remove_custom_stopwords)

# Lihat hasil
data

--2025-06-05 14:48:56--  https://raw.githubusercontent.com/ahmadizza/AMS-MID-CS-GROUP-G/refs/heads/main/stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9027 (8.8K) [text/plain]
Saving to: ‘data/stopwords_id.txt’

data/stopwords_id.t 100%[===================>]   8.82K  --.-KB/s    in 0s      

2025-06-05 14:48:57 (63.4 MB/s) - ‘data/stopwords_id.txt’ saved [9027/9027]



,published_at,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2,no_stopwords
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus,program bagus,program bagus,bagus
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...,gubernur jawa tengah tidak mikirin masa depan ...,mikirin mikirnya wudele
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan dewan perwakilan rakyat daerah dewan ...
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya ibunya anaknya kesinihormat
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...,siapa tahu akan gedenya jadi tentara nasional ...,gedenya tentara nasional bagus setuju kebijakan
...,...,...,...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...,ini manusia tidak berguna kamu bisanya mengala...,ini manusia tidak berguna kamu bisanya mengala...,manusia berguna bisanya mengalangin denerin ka...
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...,jadi anak nakal biarin saja kata yang tidak se...,jadi anak nakal biarin saja kata yang tidak se...,biarin setuju disiplinlucu inikan tujuannya se...
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...,kalau anak anak ini di siksa boleh ibu menangi...,kalau anak anak ini di siksa boleh ibu menangi...,siksa menangis sejadi didik karakter sifat dis...
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...,sudah lah hesti ngapai nagisin anak orang sila...,sudah lah hesti ngapai nagisin anak orang sila...,hesti ngapai nagisin silakan prihatin peduli t...


###Data Lemmatization

In [21]:
# Proses mengubah kata ke bentuk dasarnya (root word).
# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming menggunakan Sastrawi
def stem_text(text):
    return stemmer.stem(text)

# Terapkan stemming pada kolom 'no_stopwords'
data['stemmed_text'] = data['no_stopwords'].apply(stem_text)

data

,published_at,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2,no_stopwords,stemmed_text
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus,program bagus,program bagus,bagus,bagus
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...,gubernur jawa tengah tidak mikirin masa depan ...,mikirin mikirnya wudele,mikirin mikirnya wudele
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan dewan perwakilan rakyat daerah dewan ...,bubar dewan wakil rakyat daerah dewan wakil ra...
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya ibunya anaknya kesinihormat,takut ibu anak kesinihormat
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...,siapa tahu akan gedenya jadi tentara nasional ...,gedenya tentara nasional bagus setuju kebijakan,gede tentara nasional bagus tuju bijak
...,...,...,...,...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...,ini manusia tidak berguna kamu bisanya mengala...,ini manusia tidak berguna kamu bisanya mengala...,manusia berguna bisanya mengalangin denerin ka...,manusia guna bisa mengalangin denerin kangdedi...
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...,jadi anak nakal biarin saja kata yang tidak se...,jadi anak nakal biarin saja kata yang tidak se...,biarin setuju disiplinlucu inikan tujuannya se...,biarin tuju disiplinlucu ini tuju tuju tuanyak...
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...,kalau anak anak ini di siksa boleh ibu menangi...,kalau anak anak ini di siksa boleh ibu menangi...,siksa menangis sejadi didik karakter sifat dis...,siksa menang jadi didik karakter sifat disipli...
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...,sudah lah hesti ngapai nagisin anak orang sila...,sudah lah hesti ngapai nagisin anak orang sila...,hesti ngapai nagisin silakan prihatin peduli t...,hesti ngapai nagisin sila prihatin peduli tena...


### Stopword lanjutan

In [22]:
# --- Kode untuk Top 100 Kata dari df_stemmed ---

from collections import Counter

all_words = []

# Iterasi melalui setiap entri di kolom 'stemmed_text'
# Menggunakan .dropna() untuk menghindari error jika ada nilai NaN
# Menggunakan .astype(str) untuk mengubah semua nilai menjadi string (mengantisipasi jika ada tipe data lain)
for text_entry in data['stemmed_text'].dropna().astype(str):
    if text_entry.strip():  # Memastikan string tidak kosong setelah di-strip
        # Jika isi 'stemmed_text' adalah string kata-kata yang dipisahkan spasi
        words = text_entry.split()
        all_words.extend(words)
        # Jika isi 'stemmed_text' sudah berupa list of words, Anda bisa langsung:
        # all_words.extend(text_entry) # Namun, ini memerlukan text_entry sudah pasti list

# Hitung frekuensi setiap kata
word_counts = Counter(all_words)

# Dapatkan 100 kata paling umum
top_100_words = word_counts.most_common(100)

print("--- Top 100 kata yang sering muncul dari 'stemmed_text' ---")
if top_100_words:
    for word, count in top_100_words:
        print(f"{word}: {count}")
else:
    print("Tidak ada kata untuk dihitung (mungkin semua entri kosong atau NaN).")
print("-" * 40)

--- Top 100 kata yang sering muncul dari 'stemmed_text' ---
komisi: 622
lindung: 610
rakyat: 528
didik: 491
tuju: 463
bicara: 376
moga: 368
jabat: 358
nasional: 331
tentara: 322
masuk: 271
dukung: 246
dewan: 244
bagus: 240
sehat: 230
wakil: 227
mantap: 227
sekolah: 223
disiplin: 222
solusi: 204
umpat: 200
maju: 193
manusia: 187
hasil: 179
suka: 175
pimpin: 172
hak: 169
bijak: 167
masyarakat: 166
jalan: 164
keren: 162
baik: 160
omon: 159
warga: 154
uang: 151
kerja: 150
bina: 149
generasi: 149
teori: 148
timur: 145
antem: 139
kasih: 138
menang: 137
anak: 134
si: 134
guna: 131
kritik: 130
pikir: 127
makan: 126
nyata: 125
asasi: 125
ajar: 124
daerah: 123
hebat: 122
bubar: 121
nyinyir: 121
coba: 120
amin: 118
urus: 116
ono: 113
kirim: 108
bikin: 108
semangat: 105
karakter: 103
latih: 101
bangga: 101
salah: 101
tindak: 99
ubah: 99
muda: 99
hati: 97
televisi: 95
komentar: 94
perintah: 93
dengar: 92
wajib: 92
sila: 92
mental: 91
bilang: 91
takut: 89
mudah: 89
bawa: 88
umur: 88
cari: 88
langsun

In [23]:
# Tambahkan stopwords tambahan
additional_stopwords = {
    'adik', 'agama', 'ajar', 'ala', 'amp', 'anak', 'anggota', 'anggar', 'antem', 'apa', 'arti', 'asasi', 'bantu',
    'bawa', 'beda', 'bicara', 'bikin', 'bilang', 'bina', 'bukti', 'cari', 'coba', 'contoh', 'daerah', 'dewan',
    'diam', 'didik', 'disiplin', 'dukung', 'gaji buta', 'generasi', 'generasi muda', 'gubenur', 'hak', 'hasil',
    'henti', 'hormat', 'ide', 'ikut', 'jakarta', 'jalan', 'jaman', 'jabatan', 'jabat', 'kak seto', 'kali',
    'karakter', 'keluarga', 'kerja', 'kaya', 'kirim', 'khofifah', 'komentar', 'komisi', 'koruptor', 'lindung',
    'lihat', 'lembaga', 'lulus', 'langsung', 'malu', 'makan', 'makan gaji', 'manusia', 'masuk', 'masyarakat',
    'media', 'metropolitan', 'metropolitan televisi', 'mental', 'moga', 'mu', 'muda', 'murid', 'nama', 'nasional',
    'neng', 'negara', 'nilai', 'nol', 'nyata', 'omong', 'omon', 'omon omon', 'ono', 'otak', 'pas', 'penuh',
    'pikir', 'pilih', 'pimpin', 'pintar', 'republik', 'remaja', 'rg', 'rakyat', 'rumah', 'sila', 'sehat', 'sekolah',
    'sesuaI', 'si', 'smp', 'solusi', 'swt', 'tanggung', 'televisi', 'teman', 'tentara', 'teori', 'tinggal', 'timur',
    'tuju', 'tujuan', 'tuju', 'ubah', 'uang', 'umpat', 'umpat umpat', 'urus', 'wakil', 'wajib', 'warga', 'wudele',
    'nama', 'rocky'
}

# Gabungkan dengan stopwords yang sudah ada
id_stop = set(id_stop).union(additional_stopwords)

# Fungsi hapus stopwords dari list kata
def remove_custom_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in id_stop])

# Terapkan ke kolom hasil lemmatization (yang sudah berupa list kata)
data['no_stopwords2'] = data['stemmed_text'].apply(remove_custom_stopwords)

data

,published_at,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2,no_stopwords,stemmed_text,no_stopwords2
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus,program bagus,program bagus,bagus,bagus,bagus
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...,gubernur jawa tengah tidak mikirin masa depan ...,mikirin mikirnya wudele,mikirin mikirnya wudele,mikirin mikirnya
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan dewan perwakilan rakyat daerah dewan ...,bubar dewan wakil rakyat daerah dewan wakil ra...,bubar retorika
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya ibunya anaknya kesinihormat,takut ibu anak kesinihormat,takut kesinihormat
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...,siapa tahu akan gedenya jadi tentara nasional ...,gedenya tentara nasional bagus setuju kebijakan,gede tentara nasional bagus tuju bijak,gede bagus bijak
...,...,...,...,...,...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...,ini manusia tidak berguna kamu bisanya mengala...,ini manusia tidak berguna kamu bisanya mengala...,manusia berguna bisanya mengalangin denerin ka...,manusia guna bisa mengalangin denerin kangdedi...,mengalangin denerin kangdedi manusa sebagay te...
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...,jadi anak nakal biarin saja kata yang tidak se...,jadi anak nakal biarin saja kata yang tidak se...,biarin setuju disiplinlucu inikan tujuannya se...,biarin tuju disiplinlucu ini tuju tuju tuanyak...,biarin disiplinlucu tuanyakalu menang ledek
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...,kalau anak anak ini di siksa boleh ibu menangi...,kalau anak anak ini di siksa boleh ibu menangi...,siksa menangis sejadi didik karakter sifat dis...,siksa menang jadi didik karakter sifat disipli...,siksa menang sifat arah
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...,sudah lah hesti ngapai nagisin anak orang sila...,sudah lah hesti ngapai nagisin anak orang sila...,hesti ngapai nagisin silakan prihatin peduli t...,hesti ngapai nagisin sila prihatin peduli tena...,hesti ngapai nagisin prihatin peduli tenar hesti


In [24]:
# --- Kode untuk Top 100 Kata dari df_stemmed ---

from collections import Counter

all_words = []

# Iterasi melalui setiap entri di kolom 'stemmed_text'
# Menggunakan .dropna() untuk menghindari error jika ada nilai NaN
# Menggunakan .astype(str) untuk mengubah semua nilai menjadi string (mengantisipasi jika ada tipe data lain)
for text_entry in data['no_stopwords2'].dropna().astype(str):
    if text_entry.strip():  # Memastikan string tidak kosong setelah di-strip
        # Jika isi 'stemmed_text' adalah string kata-kata yang dipisahkan spasi
        words = text_entry.split()
        all_words.extend(words)
        # Jika isi 'stemmed_text' sudah berupa list of words, Anda bisa langsung:
        # all_words.extend(text_entry) # Namun, ini memerlukan text_entry sudah pasti list

# Hitung frekuensi setiap kata
word_counts = Counter(all_words)

# Dapatkan 100 kata paling umum
top_100_words = word_counts.most_common(100)

print("--- Top 100 kata yang sering muncul dari 'stemmed_text' ---")
if top_100_words:
    for word, count in top_100_words:
        print(f"{word}: {count}")
else:
    print("Tidak ada kata untuk dihitung (mungkin semua entri kosong atau NaN).")
print("-" * 40)

--- Top 100 kata yang sering muncul dari 'stemmed_text' ---
bagus: 240
mantap: 227
maju: 193
suka: 175
bijak: 167
keren: 162
kasih: 138
menang: 137
kritik: 130
hebat: 122
bubar: 121
nyinyir: 121
amin: 118
semangat: 105
latih: 101
bangga: 101
salah: 101
tindak: 99
hati: 97
perintah: 93
dengar: 92
takut: 89
mudah: 89
umur: 88
hidup: 84
laku: 82
tangan: 81
kaji: 80
sukses: 80
cinta: 80
alhamdulillah: 80
gaji: 79
bodoh: 79
positif: 74
manfaat: 74
terima: 73
gerung: 73
guru: 71
atur: 70
butuh: 68
peduli: 63
bentuk: 59
negeri: 59
susah: 59
keras: 58
berita: 58
mata: 57
teriak: 57
lapang: 55
habis: 55
cerdas: 55
bangun: 54
paham: 53
main: 53
harap: 50
emas: 50
rusak: 50
suruh: 50
terap: 47
protes: 47
senang: 47
provinsi: 46
politik: 46
polisi: 46
pulang: 46
dangkal: 46
korupsi: 45
lingkung: 45
buta: 44
motor: 44
sayang: 44
air: 43
kuat: 43
iri: 43
ayo: 43
duduk: 43
jiwa: 43
dasar: 42
pesantren: 42
perhati: 42
komen: 41
cocok: 41
sampah: 41
juang: 40
tidur: 40
beliau: 40
kak: 40
mulut: 39
suar

### Tokenisasi

In [25]:
# Proses memecah teks menjadi bagian-bagian kecil yang disebut token. Token biasanya berupa kata, frasa, atau simbol.
# Fungsi untuk tokenisasi teks
def tokenize_text(text):
    return word_tokenize(text)  # Tokenisasi teks menjadi list kata

# Terapkan tokenisasi ke kolom 'stemmed_text'
data['tokenize_text'] = data['no_stopwords2'].apply(tokenize_text)

data

,published_at,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2,no_stopwords,stemmed_text,no_stopwords2,tokenize_text
0,2025-05-21 10:53:25,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",Program bagus,1,program bagus,program bagus,program bagus,bagus,bagus,bagus,[bagus]
1,2025-05-21 00:28:40,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...",gub jateng ga mikirin masa depan anak.mikiri w...,2,gub jateng ga mikirin masa depan anakmikiri wu...,gubernur jawa tengah tidak mikirin masa depan ...,gubernur jawa tengah tidak mikirin masa depan ...,mikirin mikirnya wudele,mikirin mikirnya wudele,mikirin mikirnya,"[mikirin, mikirnya]"
2,2025-05-18 11:55:57,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Bubarkan saja DPRD dan DPR ini, penuh dengan r...",2,bubarkan saja dprd dan dpr ini penuh dengan re...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan saja dewan perwakilan rakyat daerah d...,bubarkan dewan perwakilan rakyat daerah dewan ...,bubar dewan wakil rakyat daerah dewan wakil ra...,bubar retorika,"[bubar, retorika]"
3,2025-05-17 23:56:01,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","TakutNya anak gak punya masalah, Tapi ibuNya y...",1,takutnya anak gak punya masalah tapi ibunya yg...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya anak tidak punya masalah tapi ibunya ...,takutnya ibunya anaknya kesinihormat,takut ibu anak kesinihormat,takut kesinihormat,"[takut, kesinihormat]"
4,2025-05-17 16:44:17,"Top Review - Solusi Anak Nakal, Masuk Barak Mi...","Siapa tau kan gedenya jadi tni, malah bagus in...",1,siapa tau kan gedenya jadi tni malah bagus ini...,siapa tahu akan gedenya jadi tentara nasional ...,siapa tahu akan gedenya jadi tentara nasional ...,gedenya tentara nasional bagus setuju kebijakan,gede tentara nasional bagus tuju bijak,gede bagus bijak,"[gede, bagus, bijak]"
...,...,...,...,...,...,...,...,...,...,...,...
6365,2025-05-04 14:01:16,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Ini manusia tidak berguna km bisanya mengalang...,1,ini manusia tidak berguna km bisanya mengalang...,ini manusia tidak berguna kamu bisanya mengala...,ini manusia tidak berguna kamu bisanya mengala...,manusia berguna bisanya mengalangin denerin ka...,manusia guna bisa mengalangin denerin kangdedi...,mengalangin denerin kangdedi manusa sebagay te...,"[mengalangin, denerin, kangdedi, manusa, sebag..."
6366,2025-05-04 14:00:50,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Jadi anak bandel biarin aja kata yg ga setuju ...,2,jadi anak bandel biarin aja kata yg ga setuju ...,jadi anak nakal biarin saja kata yang tidak se...,jadi anak nakal biarin saja kata yang tidak se...,biarin setuju disiplinlucu inikan tujuannya se...,biarin tuju disiplinlucu ini tuju tuju tuanyak...,biarin disiplinlucu tuanyakalu menang ledek,"[biarin, disiplinlucu, tuanyakalu, menang, ledek]"
6367,2025-05-04 14:00:36,Pro kontra Program Pembinaan Siswa Nakal di Ba...,Kalo anak anak ini di siksa boleh ibu nangis s...,1,kalo anak anak ini di siksa boleh ibu nangis s...,kalau anak anak ini di siksa boleh ibu menangi...,kalau anak anak ini di siksa boleh ibu menangi...,siksa menangis sejadi didik karakter sifat dis...,siksa menang jadi didik karakter sifat disipli...,siksa menang sifat arah,"[siksa, menang, sifat, arah]"
6368,2025-05-04 13:54:09,Pro kontra Program Pembinaan Siswa Nakal di Ba...,"Dah lah hesti , ngapai nagisin ank org ,, sok ...",2,dah lah hesti ngapai nagisin ank org sok iye p...,sudah lah hesti ngapai nagisin anak orang sila...,sudah lah hesti ngapai nagisin anak orang sila...,hesti ngapai nagisin silakan prihatin peduli t...,hesti ngapai nagisin sila prihatin peduli tena...,hesti ngapai nagisin prihatin peduli tenar hesti,"[hesti, ngapai, nagisin, prihatin, peduli, ten..."


## Ekspor Data Clean

In [26]:
# Simpan ke CSV (opsional)
# data.to_csv('final_preprocessed_text_Full.csv', index=False)

Data diekspor dan akan diunggah ke GitHub untuk memudahkan proses visualisasi, sehingga tidak perlu menjalankan ulang seluruh proses dari data mentah.